# M3 Reconstruction
### Training a Neural Network to identify the best M3 Combo
By Zach Shelton
4/21/2021

In [1]:
#NanoAOD HackSchema
#Solution from Danny Noonan

In [1]:
from __future__ import print_function, division
import uproot
import numpy as np
#Make sure to install both old awkward0 and new awkward1(referred to now as awkward)
import awkward1 as ak
import awkward0 as ak0
from coffea.nanoevents import NanoAODSchema,NanoEventsFactory
from uproot3_methods import TLorentzVectorArray
import uproot3_methods
import numpy as np
import coffea.hist as hist
import matplotlib.pyplot as plt
import awkward
class HackSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        base_form["contents"].pop("Electron_photonIdx", None)
        super().__init__(base_form)
def m3_recon(tree):
    comb= ak.combinations(tree,n=3,axis=1,fields=['j1','j2','j3'])
    trijets= comb.j1+comb.j2+comb.j3
    recon =ak.max(trijets,axis=1)
    reconfinal=np.sqrt(recon.t*recon.t-recon.x*recon.x-recon.y*recon.y-recon.z*recon.z)
    list1= ak.to_numpy(reconfinal)
    return list1
files ="TTbarPowheg_Semilept_Skim_NanoAOD_1of21.root"
import coffea.processor as processor
from pprint import pprint
file=uproot.open(files)
nEvents=file['hEvents'].values[0]+file['hEvents'].values[2]
from pprint import pprint

C:\Users\zshel\anaconda3\envs\top_tag1\lib\site-packages\awkward0\__init__.py:23: FutureWarning: Consider switching from 'awkward0' to 'awkward', since the new interface became the default in 2020.

    pip install -U awkward

In Python:

    >>> import awkward as ak
    >>> new_style_array = ak.from_awkward0(old_style_array)
    >>> old_style_array = ak.to_awkward0(new_style_array)

  FutureWarning


## Note: It seems the Jet columns are sorted from greatest p_t to smallest p_t
Feel free to test, but it seems to be my observation, choosing the 1st, 2nd or 3rd jet via index should remove the issue of it being a coffea sorting artifact or procedure

In [2]:
#Now lets redo with the cuts detailed by CMS Draft Analysis
#https://drive.google.com/file/d/1XEOLyZ-Q1HdEQY379RpyyQkOF1Q8KlsL/view
events =NanoEventsFactory.from_root(files,schemaclass=HackSchema).events()
events.GenPart.pdgId

<Array [[21, 21, 6, -6, ... -13, 14, 11, -11]] type='1734354 * var * int32[param...'>

In [3]:
#Condensing All Cuts to a single Cell
tight_jets=events.Jet
print(tight_jets)
jetSel = ak.num(tight_jets[((tight_jets.pt>30)&(tight_jets.eta<2.4)&(tight_jets.eta>-2.4))],axis=1)>=3
jetSelection=(jetSel&(ak.num(tight_jets.btagCSVV2>.4184)>=1))

[[Jet, Jet, Jet, Jet, Jet], [Jet, Jet, ... Jet, Jet, Jet, Jet, Jet, Jet, Jet, Jet]]


In [4]:
#Condensing_all Lepton_cuts
tight_muons = events.Muon
muonsel=ak.num(tight_muons[((tight_muons.pt>30)&(abs(tight_muons.eta)<2.4))],axis=1)==1
tight_electrons= events.Electron
electronsel=ak.num(tight_electrons[((tight_electrons.pt>35)&(abs(tight_electrons.eta)<2.4))],axis=1)==1
leptonsel=(muonsel|electronsel)
print(leptonsel)

[True, True, True, True, False, True, True, ... True, True, True, True, True, True]


In [5]:
jetlepselmask = (jetSelection&leptonsel)
print((jetlepselmask))
print(events[jetlepselmask])
final=events[jetlepselmask]
#postcuts_m3=m3_recon(events[jetlepselmask].Jet)

[True, True, True, True, False, True, True, ... True, True, True, True, True, True]
[<event 1:50992:8498517>, <event 1:50992:8498507>, ... <event 1:641843:106973897>]


In [13]:
events =NanoEventsFactory.from_root(files,schemaclass=HackSchema).events()
events.Jet.fields
#events.Jet.

['area',
 'btagCMVA',
 'btagCSVV2',
 'btagDeepB',
 'btagDeepC',
 'btagDeepFlavB',
 'btagDeepFlavC',
 'chEmEF',
 'chHEF',
 'eta',
 'jercCHF',
 'jercCHPUF',
 'mass',
 'muEF',
 'muonSubtrFactor',
 'neEmEF',
 'neHEF',
 'phi',
 'pt',
 'qgl',
 'rawFactor',
 'bRegCorr',
 'bRegRes',
 'electronIdx1',
 'electronIdx2',
 'jetId',
 'muonIdx1',
 'muonIdx2',
 'nConstituents',
 'nElectrons',
 'nMuons',
 'puId',
 'genJetIdx',
 'hadronFlavour',
 'partonFlavour',
 'cleanmask',
 'electronIdx1G',
 'electronIdx2G',
 'muonIdx1G',
 'muonIdx2G',
 'genJetIdxG',
 'muonIdxG',
 'electronIdxG']

In [15]:
jets = ak.zip({"pt":final.Jet.pt[:,0:8],"eta":final.Jet.eta[:,0:8],"phi":final.Jet.phi[:,0:8],"mass":final.Jet.mass[:,0:8],"btag":final.Jet.btagCSVV2[:,0:8]})
jets.fields

['pt', 'eta', 'phi', 'mass', 'btag']

In [65]:
#First cut Combos without b-tagged
#This will become my data tensor I pass to a Neural Net, pending additions of more(the combinations):
comb= ak.combinations(jets,n=3,axis=1,highlevel=1)
truthcomb=ak.combinations(final.GenJet.partonFlavour[:,0:8],n=3,axis=1)

In [83]:
truth={'b~':-5,'b':5,'s':3,'s~':-3,'c':4,'c~':-4,'non-jet':0,'d':1,'d~':-1,'t':6,'t~':-6,'g':21,'g~':-21,'u':2,'u~':-2}
test =truthcomb[1]
#Absolute Value of Truth Terms should add up to 11,13,15 corresponding to d,s,b

In [86]:
sumray=(abs(truthcomb['0'])+abs(truthcomb['1'])+abs(truthcomb['2']))
m1=sumray==11
m2=sumray==13
m3=sumray==15
mask=m1|m2|m3

<Array [[False, True, False, ... False, False]] type='1381189 * var * bool'>


In [88]:
#mask is a 35xN Awkward array.
TruthValues=mask
pprint(mask[1])

<Array [True, False, False, ... False, False] type='35 * bool'>


## Notes for work
- Jet TightID
- More 
- Particle Values
- Delta R

Wrap into Coffea Executor
Seperate values, weights and triggers
read these into tensors for Keras
Best ML Algorithm?
- Deep Neural Net
- Iterative Boosted Tree
    - They are fast
    - External ML algorithm modifies BDT parameters
- Combine old processes together
Should I use Keras or PyTorch?
_______________________________________________________________________________
Running Notes and questions
- Standardizing the "size", tensorflow has a ragged tensor, which is tf's variable size data arrays. I keep getting the following output
    
- Awkward doesn't have native access to ndim? That seems not correct, not sure if its my implementation. 

In [1]:
#Create truth groups 2 bjets and 1 light(gluon?)

## Sources/Further Reading:

CMS Draft Analysis Note(FIT, AAS, Wayne State, NISTR): https://drive.google.com/file/d/1XEOLyZ-Q1HdEQY379RpyyQkOF1Q8KlsL/view

Invariant Mass Wikipedia Article: https://en.wikipedia.org/wiki/Invariant_mass

Measurement of the semileptonic tt + γ production cross-section in pp collisions at √s = 8 Tev: https://arxiv.org/pdf/1706.08128.pdf

Coffea Documentation: https://coffeateam.github.io/coffea/

Awkward Array Documentation: https://awkward-array.readthedocs.io/en/latest/

AOD DATA GUIDE: https://twiki.cern.ch/twiki/bin/view/CMSPublic/SWGuideAodDataTable